
# M5 Forecasting — Quarterly Point Forecasts with Prediction Intervals (80% / 95%)

This notebook answers **"How much will we sell?"** by producing **point forecasts** and **prediction intervals** for the next **4–12 quarters** from the **M5 Forecasting (Walmart) dataset**.  
It aggregates daily item-store sales to **quarterly** at multiple levels:
- **SKU** (`item_id`) × **Region** (`state_id` and `store_id`)
- **Category/Department** (brand-like aggregation proxy)
- **Total**

> **Note on "Channel":** The M5 dataset does **not** include channels (e.g., pharmacy/retail vs derm clinics). We therefore omit channel splits here. If you have an external mapping, you can join it by `store_id` or `item_id` and re-run the same pipeline.



## Data Dictionary (core files)

- `sales_train_validation.csv`: Daily unit sales by `item_id` × `store_id` with columns `d_1…d_1913`. Includes hierarchy: `item_id`, `dept_id`, `cat_id`, `store_id`, `state_id`.
- `calendar.csv`: Maps `d_x` to actual `date`, with special event fields such as `event_name_1`, `event_type_1`, `snap_CA/TX/WI` and weekday flags.
- `sell_prices.csv`: Price per `store_id` × `item_id` × `wm_yr_wk` (weekly).
- `sales_train_evaluation.csv`: Extends the horizon for evaluation (optional for this workflow).

> Source: Kaggle “M5 Forecasting – Accuracy” data page.


In [ ]:

# --- Setup & configuration
import os
import gc
import sys
import math
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

# Modeling
from prophet import Prophet  # pip install prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Plotting (optional visual checks)
import matplotlib.pyplot as plt

# Controls
TOP_K_ITEMS_PER_STATE = 50     # for speed; increase as needed
FORECAST_QUARTERS = 8          # choose 4..12
PI_LEVELS = [0.80, 0.95]

# Reproducibility
SEED = 42
np.random.seed(SEED)

print('Versions:', 
      'pandas', pd.__version__, 
      '| prophet', Prophet.__version__ if hasattr(Prophet,'__version__') else 'n/a')


In [ ]:

# --- Load data (Kaggle paths). If running locally, adjust paths accordingly.
DATA_DIR = '/kaggle/input/m5-forecasting-accuracy'
assert os.path.exists(DATA_DIR), "Update DATA_DIR to point to the M5 data folder."

sales = pd.read_csv(f'{DATA_DIR}/sales_train_validation.csv')
calendar = pd.read_csv(f'{DATA_DIR}/calendar.csv')
prices = pd.read_csv(f'{DATA_DIR}/sell_prices.csv')

sales.shape, calendar.shape, prices.shape


In [ ]:

# --- Reshape to long daily frame and merge calendar to get real dates
id_cols = ['item_id','dept_id','cat_id','store_id','state_id']
d_cols = [c for c in sales.columns if c.startswith('d_')]

sales_long = sales.melt(id_vars=id_cols, value_vars=d_cols, var_name='d', value_name='units')
sales_long = sales_long.merge(calendar[['d','date','wm_yr_wk','weekday','wday','month','year',
                                        'event_name_1','event_type_1','event_name_2','event_type_2',
                                        'snap_CA','snap_TX','snap_WI']], on='d', how='left')
sales_long['date'] = pd.to_datetime(sales_long['date'])
sales_long = sales_long.sort_values(['item_id','store_id','date']).reset_index(drop=True)

sales_long.head()


In [ ]:

# --- Join weekly price to daily via wm_yr_wk
sales_long = sales_long.merge(prices, on=['store_id','item_id','wm_yr_wk'], how='left')
# Forward-fill price within each item-store as needed
sales_long['sell_price'] = sales_long.groupby(['item_id','store_id'])['sell_price'].apply(lambda s: s.ffill().bfill())
sales_long[['date','item_id','store_id','units','sell_price']].head()


In [ ]:

# --- Aggregate from daily to quarterly
def to_quarter(dt):
    return f"{dt.year}Q{((dt.month-1)//3)+1}"

sales_long['quarter'] = sales_long['date'].apply(to_quarter)
quarterly = (sales_long
             .groupby(['state_id','store_id','cat_id','dept_id','item_id','quarter'], as_index=False)
             .agg(units=('units','sum'),
                  avg_price=('sell_price','mean')))

# Create a proper period-end date for Prophet (use quarter end for timestamps)
quarterly['ds'] = pd.PeriodIndex(quarterly['quarter'], freq='Q').to_timestamp(how='end')
quarterly = quarterly.sort_values(['item_id','store_id','ds']).reset_index(drop=True)
quarterly.head()


In [ ]:

# --- Backtesting utilities
def rolling_backtest_prophet(df, horizon_q=FORECAST_QUARTERS, initial_q=16, step_q=1, pi=0.95):
    """
    Rolling-origin backtest for one series (df with columns ds, y).
    Returns metrics and concatenated forecasts.
    """
    cutoffs = []
    start_idx = initial_q
    while start_idx + step_q <= len(df) - horizon_q:
        cutoffs.append(df['ds'].iloc[start_idx])
        start_idx += step_q
    forecasts = []
    for cutoff in cutoffs:
        train = df[df['ds'] < cutoff].copy()
        m = Prophet(interval_width=pi)
        # Add price as a regressor if present
        if 'avg_price' in train.columns:
            m.add_regressor('avg_price')
        m.fit(train[['ds','y'] + (['avg_price'] if 'avg_price' in train.columns else [])])
        future = pd.DataFrame({'ds': pd.period_range(cutoff, periods=horizon_q, freq='Q').to_timestamp(how='end')})
        if 'avg_price' in df.columns:
            # naive forward fill of price using last train value
            last_price = train['avg_price'].iloc[-1] if not train['avg_price'].isna().all() else np.nan
            future['avg_price'] = last_price
        fcst = m.predict(future)
        fcst = fcst[['ds','yhat','yhat_lower','yhat_upper']]
        fcst['cutoff'] = cutoff
        # join actuals for error calc
        fcst = fcst.merge(df[['ds','y']], on='ds', how='left')
        forecasts.append(fcst)
    if not forecasts:
        return None, None, None, None
    fcst_all = pd.concat(forecasts, ignore_index=True)
    # Metrics
    dfm = fcst_all.dropna(subset=['y']).copy()
    if dfm.empty:
        return None, None, None, fcst_all
    mae = mean_absolute_error(dfm['y'], dfm['yhat'])
    rmse = mean_squared_error(dfm['y'], dfm['yhat'], squared=False)
    # avoid division by zero for MAPE/WAPE
    eps = 1e-8
    mape = np.mean(np.abs((dfm['y'] - dfm['yhat']) / np.where(dfm['y']==0, eps, dfm['y']))) * 100
    wape = dfm.eval('abs(y - yhat)').sum() / (np.abs(dfm['y']).sum() + eps) * 100
    # MASE with seasonal period 4 (quarters) using naive seasonal forecast
    if len(df) > 4:
        denom = np.mean(np.abs(df['y'].iloc[4:].values - df['y'].iloc[:-4].values))
        mase = (np.abs(dfm['y'] - dfm['yhat']).mean() / (denom + eps))
    else:
        mase = np.nan
    return mae, rmse, (mape, wape, mase), fcst_all


In [ ]:

# --- Modeling loop (Prophet) at item-state level for TOP_K items per state
# Select top-K SKUs by trailing-2y quarterly volume per state for speed
latest_year = quarterly['ds'].dt.year.max()
window = quarterly[quarterly['ds'] >= pd.Timestamp(latest_year-2,12,31)]
topk = (window.groupby(['state_id','item_id'], as_index=False)['units'].sum()
              .sort_values(['state_id','units'], ascending=[True,False]))
topk = topk.groupby('state_id').head(TOP_K_ITEMS_PER_STATE)

results = []
forecasts_out = []

for state in topk['state_id'].unique():
    items = topk[topk['state_id']==state]['item_id'].unique()
    for item in items:
        s = quarterly[(quarterly['state_id']==state) & (quarterly['item_id']==item)].copy()
        s = s.rename(columns={'units':'y'})
        s = s[['ds','y','avg_price']].sort_values('ds')
        if s['y'].sum() == 0 or s['y'].notna().sum() < 8:
            continue  # skip empty/very short series
        # Backtest
        mae, rmse, others, fcst_all = rolling_backtest_prophet(s, horizon_q=4, initial_q=12, step_q=1, pi=0.95)
        if others is None:
            continue
        mape, wape, mase = others
        # Train final model with 95% PI (also compute 80% by a second predict call)
        m = Prophet(interval_width=0.95)
        m.add_regressor('avg_price')
        m.fit(s[['ds','y','avg_price']])
        future = pd.DataFrame({'ds': pd.period_range(s['ds'].max()+pd.offsets.QuarterEnd(), periods=FORECAST_QUARTERS, freq='Q').to_timestamp(how='end')})
        future['avg_price'] = s['avg_price'].iloc[-1]
        fcst95 = m.predict(future)[['ds','yhat','yhat_lower','yhat_upper']].rename(columns={'yhat_lower':'pi95_lo','yhat_upper':'pi95_hi'})
        # 80% intervals via a new model with interval_width=0.80 (reuse fit by setting width and predicting again is not supported, so refit quickly)
        m80 = Prophet(interval_width=0.80)
        m80.add_regressor('avg_price')
        m80.fit(s[['ds','y','avg_price']])
        fcst80 = m80.predict(future)[['ds','yhat_lower','yhat_upper']].rename(columns={'yhat_lower':'pi80_lo','yhat_upper':'pi80_hi'})
        fcst = fcst95.merge(fcst80, on='ds')
        fcst['state_id'] = state
        fcst['item_id'] = item
        forecasts_out.append(fcst)
        results.append({'state_id':state,'item_id':item,'MAE':mae,'RMSE':rmse,'MAPE%':mape,'WAPE%':wape,'MASE':mase})

metrics = pd.DataFrame(results).sort_values(['state_id','RMSE'])
forecast_item_state = pd.concat(forecasts_out, ignore_index=True) if forecasts_out else pd.DataFrame()
metrics.head(), forecast_item_state.head()


In [ ]:

# --- Join hierarchy so we can aggregate forecasts to higher levels (dept/cat, total)
keys = sales[['item_id','dept_id','cat_id','store_id','state_id']].drop_duplicates()
forecast_item_state = forecast_item_state.merge(keys[['item_id','dept_id','cat_id']], on='item_id', how='left')

# Quarterly SKU x State forecasts are already quarterly timestamps in `ds`.
# Create higher-level aggregates by summing yhat and combining intervals via square-root of summed variances approximation.
def agg_intervals(df, group_cols):
    # Approximate by summing means and assuming independence for variance (PI width scaling)
    out = df.groupby(group_cols + ['ds'], as_index=False).agg(
        yhat=('yhat','sum'),
        pi95_lo=('pi95_lo','sum'),
        pi95_hi=('pi95_hi','sum'),
        pi80_lo=('pi80_lo','sum'),
        pi80_hi=('pi80_hi','sum'),
    )
    # Clip negatives for lower bounds
    for c in ['pi95_lo','pi80_lo']:
        out[c] = out[c].clip(lower=0.0)
    return out

fc_dept_state = agg_intervals(forecast_item_state, ['state_id','dept_id'])
fc_cat_state  = agg_intervals(forecast_item_state, ['state_id','cat_id'])
fc_total_state = agg_intervals(forecast_item_state, ['state_id'])
fc_total = agg_intervals(forecast_item_state, [])

fc_total.head()


In [ ]:

# --- Export CSVs
out_dir = '/kaggle/working/m5_quarterly_outputs'
os.makedirs(out_dir, exist_ok=True)

metrics.to_csv(f'{out_dir}/metrics_item_state_backtest.csv', index=False)
forecast_item_state.to_csv(f'{out_dir}/forecast_item_state_quarterly.csv', index=False)
fc_dept_state.to_csv(f'{out_dir}/forecast_dept_state_quarterly.csv', index=False)
fc_cat_state.to_csv(f'{out_dir}/forecast_cat_state_quarterly.csv', index=False)
fc_total_state.to_csv(f'{out_dir}/forecast_total_state_quarterly_by_state.csv', index=False)
fc_total.to_csv(f'{out_dir}/forecast_total_quarterly.csv', index=False)

print('Saved to', out_dir)


In [ ]:

# --- Preview "How much will we sell?" — Total level for next 8 quarters
preview = fc_total.sort_values('ds').groupby('ds', as_index=False).agg(
    yhat=('yhat','sum'),
    pi80_lo=('pi80_lo','sum'), pi80_hi=('pi80_hi','sum'),
    pi95_lo=('pi95_lo','sum'), pi95_hi=('pi95_hi','sum'),
)
preview.head(12)
